In [ ]:
#A = gov
#B = anmawon
#total = union(A,B)
#safe_shop = intersection(total, B)

In [1]:
path_union_shop = "./anmawon/local_update.csv"
path_gov_shop = "./government/gov_geocoded_manual_edit.csv"

import pandas as pd
import math

In [2]:

gov_shop = pd.read_csv(path_gov_shop)
anmawon_union_shop = pd.read_csv(path_union_shop)


del anmawon_union_shop["region"]
anmawon_union_shop["checked"] = True


unique_addr_list = list(set(anmawon_union_shop.loc[:, "NAVER_addr"]).union(set(gov_shop.loc[:, "NAVER_addr"])))


In [3]:
final_data_list = []
for unique_addr in unique_addr_list:
    if (gov_shop['NAVER_addr'].eq(unique_addr)).any():
        row = gov_shop.loc[gov_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        # check union(gov, anmawon)
        if (anmawon_union_shop['NAVER_addr'].eq(unique_addr)).any():
            row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
            link, checked = row["link"], "O"
            if isinstance(phone, float):
                if math.isnan(phone):
                    phone = row["phone"]
        else:
            link, checked = "", "X"

    else:  # only in anmawon_union_shop
        row = anmawon_union_shop.loc[anmawon_union_shop['NAVER_addr'] == unique_addr].to_dict(orient="records")[0]
        name, phone, x, y = [row.get(key) for key in ["name", "phone", "x", "y"]]
        link, checked = row["link"], "???(정부사이트X, 협회사이트O)"

    final_data_list.append(
        {'unique_addr': unique_addr,
        'name': name,
        'phone':phone,
        'x':x ,
        'y':y ,
        'link': link ,
        'checked':checked })

final_data = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {'unique_addr': 'str', 'name': 'str', 'phone': 'str', 'x':"float", 'y':'float', 'checked':'str', 'link':'str'}.items()})

final_data = final_data.append(final_data_list, ignore_index=False, sort=False)


/tmp/ipykernel_6856/2610851832.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append(final_data_list, ignore_index=False, sort=False)


In [6]:
with open("final_data.js", "w", encoding="utf-8") as f:
    as_txt = final_data.to_json(orient="records", force_ascii=False)

    f.write("const localtion_data = "  + as_txt)
#    as_txt = final_data.to_json(orient="records", force_ascii=False)
    # final_data.to_json(f, orient="records", force_ascii=False)

    # then, manually change to .js format as "var data = [....]"

const localtion_data = [{"unique_addr":"대전광역시 유성구 온천로 34 유성빌딩","name":"킹안마시술소","phone":"042-825-8599","x":127.3412472,"y":36.3552101,"checked":"X","link":""},{"unique_addr":"경기도 성남시 분당구 황새울로258번길 32 코리아나빌딩","name":"쿠라시온안마원","phone":"031-714-8897","x":127.1160923,"y":37.3794307,"checked":"O","link":"http:\/\/www.anmawon.com\/FindShop\/Detail?shopId=031-0246"},{"unique_addr":"부산광역시 부산진구 가야대로510번길 84","name":"가야안마지압원","phone":"991-4638","x":129.0283136,"y":35.1492892,"checked":"O","link":"http:\/\/xn--hz2b11ew7coa499cy81avya.kr\/skin\/sub_page.php?page_idx=141"},{"unique_addr":"경기도 포천시 영북면 영북로 217","name":"대산안마원","phone":"031-532-0982","x":127.273152,"y":38.0936744,"checked":"???(정부사이트X, 협회사이트O)","link":"http:\/\/www.anmawon.com\/FindShop\/Detail?shopId=031-0130"},{"unique_addr":"서울특별시 강서구 강서로 349","name":"발산안마원","phone":"02-2661-1375","x":126.8374016,"y":37.5568905,"checked":"???(정부사이트X, 협회사이트O)","link":"http:\/\/www.anmawon.com\/FindShop\/Detail?shopId=02-0003"},{"unique_addr":"경기도 안양시 